In [36]:
import refinitiv.dataplatform as rdp
import pandas as pd
import json
import datetime as dt
import math
from dateutil.relativedelta import relativedelta
from enum import Enum

### Create a session into the platform

Provide the required credentials below.

In [37]:
rdp.open_platform_session(
    "<YOUR APP KEY>",
    rdp.GrantPassword(
        username = "<YOUR MACHINE ID>",
        password = "<PASSWORD>"
    )
)

## Option Interface
The Option class utilizes the IPA Option endpoint to construct the desired request properties to price an option.  The interface supports both the ETI and OTC specifications to request from the platform the pricing values, including delta, to drive our strategies.

In [38]:
class IPAOption:
    # The IPA Contract interface allows the specification of an ETI (Exchange-Traded-Instrument) to price an 
    # existing option traded within the market.
    def define_eti(option, buysell):
        return {
            "instrumentType": "Option",
            "instrumentDefinition": {
                "underlyingType": "Eti",            
                "instrumentTag": option,
                "instrumentCode": option,
                "buySell": buysell,
            },
            "pricingParameters": {
                "underlyingTimeStamp": "Close",
            }        
        }

    # The IPA Contract interface allows the specification of an OTC (Over-The-Counter) definition to model different 
    # scenarios to price options based on standard properties such as the strike and expiry.  In addition, the interface
    # supports other modeling properties such as price, valuation date and other capabilities to model what-if scenarios 
    # to estimate the Greeks.
    def define_otc(key, buysell, callput, underlying, underlyingPrice, strike, daysToExpire, exerciseStyle, daysInFuture):
        return {
            "instrumentType": "Option",
            "instrumentDefinition": {
                "underlyingType": "Eti",            
                "instrumentTag": key,
                "exerciseStyle": exerciseStyle,
                "strike": strike,
                "endDate": (dt.datetime.today() + dt.timedelta(days=daysToExpire)).strftime("%Y-%m-%d"),
                "buySell": buysell,
                "callPut": callput,
                "underlyingDefinition": {
                    "instrumentCode": underlying
                }
            },
            "pricingParameters": {
                "underlyingPrice": underlyingPrice,
                "valuationDate": (dt.datetime.today() + dt.timedelta(days=daysInFuture)).strftime("%Y-%m-%d"),
                "underlyingTimeStamp": "Close",
                "volatilityType": "SVISurface"
            }
        }

    # Price the specified options (ETI or OTC) and register interest the fields to be included in the response.
    def price_options(self, options):
        request = {
            "fields": ["InstrumentTag", "InstrumentCode", "ExerciseType", "ValuationDate", 
                       "EndDate", "StrikePrice", "OptionPrice", "DeltaPercent", "UnderlyingRIC", 
                       "UnderlyingPrice", "ExerciseStyle", "ErrorMessage"], 
            "universe": options        
        }

        return self.endpoint.send_request(method = rdp.Endpoint.RequestMethod.POST, body_parameters = request)

    # Convert the native response into a dataframe-friendly container useful for display
    def extract_values(response):
        if response.is_success:
            headings = [header['name'] for header in response.data.raw["headers"]]
            return pd.DataFrame.from_records(response.data.raw['data'], columns=headings)
        else:
            print(json.dumps(response.status, indent=3))
            return None

    # Instantiate an IPAOption object defining the IPA endpoint to manage our Option requests.
    def __init__(self):
        self.endpoint = rdp.Endpoint(rdp.get_default_session(), 
                                     "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")

## Option Chain
The Option Chain class utilizes the Chain and Snapshot interfaces to retrieve the complete option chain for a specified instrument.  The algorithm provides the user to
filter out contracts based on the option expiry as well as a percentage Around-The-Money for the strike.  A typical option chain can contain thousands of records and the filter is a way to narrow down a specific subset.

In [39]:
class OptionType(Enum):
    NONE = 0,
    CALL = 1,
    PUT = 2

class OptionChain:
    def get_contracts(self, chain, pct_around_money, expiry):
        option_chain = []
        calls = []
        puts = []

        response = self.endpoint.send_request(query_parameters = {"universe": chain})
        if response.is_success:
            # Extract the constituents within the chain
            constituents = response.data.raw["data"]["constituents"]

            # Store within a list and filter out those based on the expiry / strike specification
            for constituent in constituents:
                # An Option Chain has the 1st constituent as the underlying - we can use this to extract our root
                if self.root == None:
                    self.underlying = constituent
                    split = self.underlying.split('.')
                    if len(split) > 1:
                        self.root = split[0]
                        self.underlying_price = self.get_price(self.underlying)
                    else:
                        print(f'Specified chain item {chain} may not be an option chain.  Ignoring.')
                        break
                else:
                    # Validate the length of the constituent - a valid constituent should be at least: (len(root) + 10) bytes.
                    if len(constituent) < len(self.root)+10:
                        print(f'Specified chain item {chain} may not be an option chain.  Ignoreing.')
                        break
                    
                    # The variance is a measure how far Around-The-Money the strike price should be filtered.
                    variance = self.underlying_price * pct_around_money

                    keep, type = self.keep_constituent(constituent, expiry, variance)
                    if keep:
                        if type == OptionType.CALL:
                            calls.append(constituent)
                        elif type == OptionType.PUT:
                            puts.append(constituent)

            for put in puts:
                option_chain.append(put)

            for call in calls:
                option_chain.append(call)

        return option_chain

    # get_price
    # Retrieve the price for the specified ric.  The price retrieved will be based on the current price.  If that 
    # is unavailable, the historical price will be returned.
    def get_price(self, ric):
        snapshot = rdp.get_snapshot(universe = [ric], fields = ['TRDPRC_1','HST_CLOSE'])
        result = snapshot['TRDPRC_1'][0];
        if math.isnan(result):
            result = snapshot['HST_CLOSE'][0]

        return result

    # keep_constituent
    # Based on the filtering parameters, expiry and variance of the strike price, determine if 
    # the specified constituent is one that we want to keep.
    #
    # A proper constituent uses the following format:
    #
    # <root><month code><day><year><strike>.U  where:
    #      <root>          - Root of our underlying, Eg: AAPL
    #      <month code>    - Single character expiration month code, Eg: 'D' (April Call)
    #      <day>           - 2 digit expiration day, Eg: 15
    #      <year>          - 2 digit expiration year, Eg: 21  (2021)
    #      <strike>        - 5 digit expiration XXX.XX, Eg: 255000 (255.00)
    #
    def keep_constituent(self, constituent, expiry, variance):
        # *********************************************************
        # Parse constituent name for the parts and apply filters
        # *********************************************************
        option_type = OptionType.NONE

        # First, determine the root
        index = constituent.find(self.root)
        if index < 0: return (False, option_type)
        index += len(self.root)

        # Next, get the month code
        month_code = constituent[index]
        index += 3

        # Then the year
        year = constituent[index:index+2]

        # Verify if the month/year is our expiry
        month, option_type = self.get_month(month_code)
        if f'{expiry:%y%m}' != f'{year}{month:02}':
            return (False, option_type)
        index += 2

        # Retrieve the strike and verify if we're Around-The-Money
        strike = float(constituent[index:index+5]) / 100;

        # Filter out the strikes, based on the type of contract
        if option_type == OptionType.PUT:
            if strike > self.underlying_price or strike < self.underlying_price - variance:
                return (False, option_type)
        elif option_type == OptionType.CALL:
            if strike < self.underlying_price or strike > self.underlying_price + variance:
                return (False, option_type)

        # We found a constituent for our option chain
        return (True, option_type)

    # get_month
    # Retrieve the actual month, based on the month code.  In addition, determine the type of option based
    # on the month code.
    def get_month(self, month_code):
        type = OptionType.CALL if month_code < 'M' else OptionType.PUT

        if month_code == 'A' or month_code == 'M': return (1, type)
        if month_code == 'B' or month_code == 'N': return (2, type)
        if month_code == 'C' or month_code == 'O': return (3, type)
        if month_code == 'D' or month_code == 'P': return (4, type)
        if month_code == 'E' or month_code == 'Q': return (5, type)
        if month_code == 'F' or month_code == 'R': return (6, type)
        if month_code == 'G' or month_code == 'S': return (7, type)
        if month_code == 'H' or month_code == 'T': return (8, type)
        if month_code == 'I' or month_code == 'U': return (9, type)
        if month_code == 'J' or month_code == 'V': return (10, type)
        if month_code == 'K' or month_code == 'W': return (11, type)
        if month_code == 'L' or month_code == 'X': return (12, type)

        return (0, OptionType.NONE)

    # Convert the native response into a dataframe-friendly container useful for display
    def convert_to_df(response):
        if response.is_success:
            headings = ["1", "Option", "Type", "2", "Expiry", "Strike", "Option Price", "Delta", "Underlying", "3", "Style", "4"]
            df = pd.DataFrame.from_records(response.data.raw['data'], columns=headings)
            df.drop(["1", "2", "3", "4"], axis=1, inplace=True)
            return df
        else:
            print(json.dumps(response.status, indent=3))
            return None

    # Instantiate an Option Chain object
    def __init__(self):
        self.root = None
        self.underlying = None
        self.underlying_price = 0.0
        self.endpoint = rdp.Endpoint(rdp.get_default_session(), "/data/pricing/beta3/views/chains")

## Delta Neutral Interface
The Delta Neutral class interface supports our use case to demonstrate how to utilize delta to perform a dynamic delta strategy.  The interface simulates the management of a portfolio where positions are added in an attempt hedge the strategy to achieve a net delta position of zero (0).  The engine supports the ability to predict the Greek values based on typical movements in the market such as changes in the underlying price as well as how values are affected as the options decay. 

In [40]:
class DeltaNeutral:
    # Locate the an option within the portfolio, by key.
    def find_position(self, key):
        for position in self.portfolio:
            if (('Key' in position) and (position['Key'] == key)): 
                return position

    # Prepare the portfolio into a dataframe suitable for console display
    def format_portfolio(self):
        data = []
        for position in self.portfolio:
            data.append([position['Instrument'], position['UnderlyingPrice'], position['Contracts'], 
                         position['Type'], f'{position["DeltaPct"]:.3}'])
        
        return pd.DataFrame(data, columns=["Instrument", "Close", "Shares", "Position", "Delta"])

    # Based on the Delta value(s) derived from the Analytics engine, balance the positions based on our exposure.
    def balance_portfolio(self, exposure):
        data = []
        initialPosition = 0
        positions = 0
        netDelta = 0
        positionDelta = 0

        for position in self.portfolio:
            if (position['Contracts'] > 0):
                positionDelta = position['DeltaPct'] * position['Contracts'] * (100 if position['DeltaPct'] < 1 else 1)
                initialPosition += positionDelta
                positions += 1
            else:
                contractPosition = (initialPosition * (1 - exposure)) / (len(self.portfolio) - positions)
                contracts = abs(round(contractPosition / (position['DeltaPct'] * 100)))
                position['Contracts'] = contracts
                positionDelta = round(position['DeltaPct'] * contracts * 100)

            data.append([position['Instrument'], position['Strike'] if 'Strike' in position else '', position['Type'], 
                         f'{position["DeltaPct"]:.3}', position['Contracts'], f'{positionDelta:6.0f}'])
            netDelta += positionDelta

        # Summary
        data.append(["","","","","Net Position Delta", f'{netDelta:6.0f}'])
        
        return pd.DataFrame(data, columns=["Instrument", "Strike", "Position", "Delta", "Contracts Traded", "Position Delta"])

    # Price the requested OTC options and update the portfolio (balance if possible).  Prepare the portfolio for display.
    def update_portfolio(self, otcOption):
        # Price the OTC options
        response = self.option.price_options([otcOption])
    
        # Extract the results
        df = IPAOption.extract_values(response)
    
        # Update our portfolio
        data = df.loc[0]

        buysell = otcOption['instrumentDefinition']['buySell']
        expiry = dt.datetime.strptime(data['EndDate'], "%Y-%m-%dT%H:%M:%SZ")
        days = (expiry - dt.datetime.strptime(data['ValuationDate'], "%Y-%m-%dT%H:%M:%SZ")).days
        key = data['InstrumentTag']
        type = "{}d {} {}".format(days, 'Long' if buysell == 'Buy' else 'Short', data['ExerciseType'])

        position = self.find_position(key)
        if ( position == None):
            underlying = data['UnderlyingRIC']
            strike = data['StrikePrice']
            callput = data['ExerciseType']

            self.portfolio.append( {
                "Key": key,
                "Underlying": underlying,
                "Instrument": "OTC:{}.{}{:08.5}".format(underlying, callput[0], strike),
                "Type": type,
                "ExerciseStyle": data['ExerciseStyle'],
                "Strike": strike,
                "BuySell": buysell,
                "CallPut": callput,
                "Expiry": expiry.strftime("%Y-%m-%d"),
                "DeltaPct": data['DeltaPercent'],
                "DaysToExpiry": days,
                "Contracts": 0
            } )
        else:
            position['Type'] = type
            position['DeltaPct'] = data['DeltaPercent']

        return self.balance_portfolio(0)

    # Model an existing position based on a new price and number of days in the future
    def model_position(self, key, underlyingPrice, daysInFuture):
        position = self.find_position(key)
        if ( position != None ):
            return self.update_portfolio(IPAOption.define_otc(key, position['BuySell'], position['CallPut'], position['Underlying'],
                                                              underlyingPrice, position['Strike'], position['DaysToExpiry'], 
                                                              position['ExerciseStyle'], daysInFuture) )

    # Instantiate a DeltaNeutral Use Case object by defining an initial position of Long 500 shares of the specified underlying.
    def __init__(self, option, underlying, underlyingPrice):
        self.option = option
        self.portfolio = [ {
                "Instrument": underlying,
                "UnderlyingPrice": underlyingPrice,
                "Type": "Long Shares",
                "DeltaPct": 1.0,
                "Contracts": 500
            }]

## Use Cases
Using the IPA Option interface, the following strategies have been defined to demonstrate the capabilities of the Analytics engine:
 - Long Strangle
 - Delta Neutral

### Long Strangle
The Long Strangle is based on selecting 2 legs within an Option Chain with the same expiry, with each leg a little bit out-of-the money.

In [12]:
# Step 1 - Retrieve our option chain.
chain = OptionChain()
contracts = chain.get_contracts("0#AAPL*.U", 0.30, dt.datetime.today() + relativedelta(months=+7))

In [13]:
print(f'Retrieved a total of {len(contracts)} contracts for the underlying {chain.underlying} with the last trade at: {chain.underlying_price}')

Retrieved a total of 24 contracts for the underlying AAPL.O with the last trade at: 313.14


In [14]:
# Step 2 - Price the selected option contracts.
eti_contracts = []
for contract in contracts:
    eti_contracts.append(IPAOption.define_eti(contract, "Buy"))

ipa_option = IPAOption()
response = ipa_option.price_options(eti_contracts)
df = OptionChain.convert_to_df(response)

In [15]:
# Step 3 - Display our option chain.  We use our chain to select our legs to build out the strategy.
df

,Option,Type,Expiry,Strike,Option Price,Delta,Underlying,Style
0,AAPLX182022000.U,PUT,2020-12-18T00:00:00Z,220.0,6.65,-0.109794,AAPL.O,AMER
1,AAPLX182023000.U,PUT,2020-12-18T00:00:00Z,230.0,7.90,-0.130280,AAPL.O,AMER
2,AAPLX182024000.U,PUT,2020-12-18T00:00:00Z,240.0,9.55,-0.155152,AAPL.O,AMER
3,AAPLX182025000.U,PUT,2020-12-18T00:00:00Z,250.0,13.10,-0.201563,AAPL.O,AMER
4,AAPLX182026000.U,PUT,2020-12-18T00:00:00Z,260.0,15.70,-0.235225,AAPL.O,AMER
5,AAPLX182027000.U,PUT,2020-12-18T00:00:00Z,270.0,18.45,-0.271170,AAPL.O,AMER
6,AAPLX182028000.U,PUT,2020-12-18T00:00:00Z,280.0,21.55,-0.310262,AAPL.O,AMER
7,AAPLX182029000.U,PUT,2020-12-18T00:00:00Z,290.0,25.15,-0.352512,AAPL.O,AMER
8,AAPLX182030000.U,PUT,2020-12-18T00:00:00Z,300.0,29.20,-0.397270,AAPL.O,AMER
9,AAPLX182030500.U,PUT,2020-12-18T00:00:00Z,305.0,31.50,-0.420336,AAPL.O,AMER


### Delta Neutral
In this use case, I will demonstrate the power of the Analytics engine by modeling a couple of different scenarios based on common changes in the market.  The intention of this strategy is to ensure the portfolio is close to being perfectly hedged.  This is done in many ways, but I will only use one simple approach for demonstration purposes.  

The purpose of this use case isn't to show the strategy around how to hedge but rather show how you can use the Analytics engine to model different scenarios which may be useful in building your positions within a portfolio.

To set up my portfolio, I will be utilizing the OTC capabilities within the IPA Option interface:
 - Start with an initial position of Long 500 shares
 - To hedge this position, I will write a call option

#### Setup
For simplicity, I decided to use the same underlying details used above.

In [49]:
underlying = chain.underlying
underlying_price = chain.underlying_price
style = "AMER" #"df.loc[0, "Style"]"
print(underlying, style, underlying_price)

AAPL.O AMER 313.14


In [50]:
# Create our Delta Neutral interface to manage our portfolio and balancing helper methods.
dneutral = DeltaNeutral(ipa_option, underlying, underlying_price)
df = dneutral.format_portfolio()
df

,Instrument,Close,Shares,Position,Delta
0,AAPL.O,313.14,500,Long Shares,1.0


#### Add a new position
To hedge my current position, I will write some call options, a little bit out-of-the money

In [51]:
strike = underlying_price * 1.075
days_to_expiry = 360
strike, days_to_expiry

(336.6255, 360)

In [52]:
df = dneutral.update_portfolio(IPAOption.define_otc("tag1", "Sell", "CALL", underlying, underlying_price, 
                                                    strike, days_to_expiry, style, 0))
print('The following strategy can be implemented:\n')
df

The following strategy can be implemented:



,Instrument,Strike,Position,Delta,Contracts Traded,Position Delta
0,AAPL.O,,Long Shares,1.0,500,500
1,OTC:AAPL.O.C00336.63,336.625,360d Short CALL,-0.401,12,-481
2,,,,,Net Position Delta,19


#### Market Conditions change
In the above portfolio, to move to a net position delta close to zero, I wrote a specific amount of call options based on the delta.  As we move forward in time, we would expect market conditions will change.  I will model a scenario where in 60 days, the underlying price will increase by 5%.  What will our portfolio look like then?

Under these conditions, because our Call option is decaying, coupled with the underlying price movement, our delta has changed, thus our overall net position has changed.

In [53]:
underlying_price *= 1.05
days_in_future = 60
df = dneutral.model_position("tag1", underlying_price, days_in_future)
print(f'\nWith a simulated price increase to {underlying_price} and {days_in_future} days in the future:\n')
df


With a simulated price increase to 328.797 and 60 days in the future:



,Instrument,Strike,Position,Delta,Contracts Traded,Position Delta
0,AAPL.O,,Long Shares,1.0,500,500
1,OTC:AAPL.O.C00336.63,336.625,300d Short CALL,-0.49,12,-588
2,,,,,Net Position Delta,-88


#### Add a new position
To re-balance our portfolio, I will now short some Put option with a strike 5% out-of-the money.

In [54]:
strike = underlying_price * 0.95
df = dneutral.update_portfolio(IPAOption.define_otc("tag2", "Sell", "PUT", underlying, underlying_price, 
                                                    strike, days_to_expiry, style, days_in_future))
print('\nAdding a new position with a strike 5% out-of-the money:\n')
df


Adding a new position with a strike 5% out-of-the money:



,Instrument,Strike,Position,Delta,Contracts Traded,Position Delta
0,AAPL.O,,Long Shares,1.0,500,500
1,OTC:AAPL.O.C00336.63,336.625,300d Short CALL,-0.49,12,-588
2,OTC:AAPL.O.P00312.36,312.357,300d Short PUT,0.349,3,105
3,,,,,Net Position Delta,17


In [58]:
rdp.close_session()